In [3]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from glob import glob
from PIL import Image
from tqdm import tqdm

### Loading image list

The image list will be used by the batch function to load the files and prepare the batch with the input data and labels

In [36]:
img_list = pd.read_csv("../../datasets/image_classification/oxford-iiit-pet/annotations/list.txt",
                       sep = " ", comment = "#", header = None,
                      names = ["image_file", "class_id", "species", "breed_id"])
img_list.loc[:, "image_file"] = img_list.loc[:, "image_file"].apply(lambda x: x + ".jpg")
img_list.head()

,image_file,class_id,species,breed_id
0,Abyssinian_100.jpg,1,1,1
1,Abyssinian_101.jpg,1,1,1
2,Abyssinian_102.jpg,1,1,1
3,Abyssinian_103.jpg,1,1,1
4,Abyssinian_104.jpg,1,1,1


### Creating the class dictionary

This class dictionary will be used to print the class string

In [37]:
class_dict = img_list.loc[:,["class_id", "image_file", "species"]]
class_dict.rename(columns = {"class_id": "class", "image_file": "breed"}, inplace = True)
class_dict.loc[:, "breed"].replace("_([0-9]{1,})\.jpg", "", regex = True, inplace = True)
class_dict.loc[:, "breed"].replace("_", " ", regex = True, inplace = True)
class_dict.loc[:, "species"].replace(1, "cat", inplace = True)
class_dict.loc[:, "species"].replace(2, "dog", inplace = True)
class_dict.drop_duplicates(keep = "first", inplace = True)
class_dict.set_index('class', inplace = True)
class_dict = class_dict.to_dict('index')
class_dict

{1: {'breed': 'Abyssinian', 'species': 'cat'},
 2: {'breed': 'american bulldog', 'species': 'dog'},
 3: {'breed': 'american pit bull terrier', 'species': 'dog'},
 4: {'breed': 'basset hound', 'species': 'dog'},
 5: {'breed': 'beagle', 'species': 'dog'},
 6: {'breed': 'Bengal', 'species': 'cat'},
 7: {'breed': 'Birman', 'species': 'cat'},
 8: {'breed': 'Bombay', 'species': 'cat'},
 9: {'breed': 'boxer', 'species': 'dog'},
 10: {'breed': 'British Shorthair', 'species': 'cat'},
 11: {'breed': 'chihuahua', 'species': 'dog'},
 12: {'breed': 'Egyptian Mau', 'species': 'cat'},
 13: {'breed': 'english cocker spaniel', 'species': 'dog'},
 14: {'breed': 'english setter', 'species': 'dog'},
 15: {'breed': 'german shorthaired', 'species': 'dog'},
 16: {'breed': 'great pyrenees', 'species': 'dog'},
 17: {'breed': 'havanese', 'species': 'dog'},
 18: {'breed': 'japanese chin', 'species': 'dog'},
 19: {'breed': 'keeshond', 'species': 'dog'},
 20: {'breed': 'leonberger', 'species': 'dog'},
 21: {'breed